In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

# Langsmith Tracing
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [2]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from pprint import pprint

llm = ChatGroq(model='groq/compound-mini')
llm

c:\Users\rahul\anaconda3\envs\genai\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022F43D43470>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022F43E8A540>, model_name='groq/compound-mini', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from  langchain_core.output_parsers import StrOutputParser 
res = llm.invoke('Can you explain me about rules of f1 in 10 lines')
parser = StrOutputParser()
for line in parser.invoke(res).split("\n"):
    print(line)

Here are the basic rules of F1 in 10 lines:

1. The Formula 1 World Championship consists of multiple Grands Prix races.
2. Each team has two drivers competing in each race.
3. The objective is to complete the most laps in the shortest time.
4. Cars must meet technical regulations and weigh at least 733 kg.
5. Drivers start from a grid position determined by qualifying laps.
6. The driver with the fastest lap in qualifying gets pole position.
7. A standard F1 race consists of 50-70 laps, depending on the circuit.
8. Drivers can make pit stops to change tires and refuel (not allowed in some races).
9. Overtaking is allowed, but drivers must follow rules to avoid collisions.
10. Points are awarded to the top 10 finishers: 25, 18, 15, 12, 10, 8, 6, 4, 2, 1.


### HumanMessages & System Messages

In [11]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
                SystemMessage(content="Translate the user message to French"),
                HumanMessage(content="You look beautiful tonight!")
]

response = llm.invoke(messages)
response.content

"Tu as l'air magnifique ce soir !"

In [13]:
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()
str_parser.invoke(response)

"Tu as l'air magnifique ce soir !"

In [14]:
# Using LCEL For Chaining

chain1 = llm|str_parser
chain1.invoke(messages)

"You look beautiful tonight translates to:\n\nTu as l'air magnifique ce soir!\n\nor, in a more formal tone:\n\nVous êtes magnifique ce soir!"

In [20]:
from langchain_core.prompts import ChatPromptTemplate

gen_prompt  = "Translate the following into {language}"
prompt = ChatPromptTemplate.from_messages([
                ('system',gen_prompt),
                ("user", "{text}")
])

prompt.invoke({"language":"French","text":"Rahul is a really amazing person"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Rahul is a really amazing person', additional_kwargs={}, response_metadata={})])

In [21]:
chain2  = prompt|llm|str_parser
chain2.invoke({"language":"French","text":"Rahul is a really amazing person"})

'To translate the sentence "Rahul is a really amazing person" into French, I will use my language capabilities.\n\nThe translation is: "Rahul est une personne vraiment incroyable."\n\nHere\'s a breakdown of the translation:\n\n- "Rahul" remains the same as it\'s a proper noun.\n- "is" is translated to "est".\n- "a" is translated to "une".\n- "really" is translated to "vraiment".\n- "amazing" is translated to "incroyable".\n- "person" is translated to "personne".'

In [9]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

llm.invoke([
    HumanMessage("Hi Im Rahul, Im a great AI Engineer"),
    AIMessage("Hello Rahul! Nice to meet you! It's great to hear that you're an AI engineer. What kind of AI projects have you been working on lately"),
    HumanMessage("Hey whats my name and what do I Do?")
])




AIMessage(content="Your name is Rahul, and you mentioned you're a great AI Engineer!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 439, 'total_tokens': 468, 'completion_time': 0.065011, 'prompt_time': 0.035566, 'queue_time': 0.524294, 'total_time': 0.100577}, 'model_name': 'groq/compound-mini', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e2d968fe-98db-4a86-a8f3-fc8ab25a5e05-0', usage_metadata={'input_tokens': 439, 'output_tokens': 29, 'total_tokens': 468})

### Chat Message History

In [25]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser

store = {}

def get_session_history(session_id :str)->BaseChatMessageHistory: # This is the base message history
    if session_id not in store:
        store[session_id] = ChatMessageHistory() # This is the active session history
    return store[session_id]

with_chat_message_history = RunnableWithMessageHistory(llm,get_session_history)

config = {"configurable":{"session_id":"chat1"}}

response = with_chat_message_history.invoke([HumanMessage(content="Hi My Name is Rahul")],
                                 config=config)

op_parse = StrOutputParser()
op_parse.invoke(response)

"Hello Rahul! It's nice to meet you. Is there something I can help you with or would you like to chat?"